In [1]:
import sys
sys.path.append('/source/main')

In [13]:
import numpy as np

import tensorflow as tf
from tensor2tensor import problems
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import registry

from data_for_train import auto_comment_problem

In [3]:
# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

In [4]:
my_problem = problems.problem("auto_comment_problem")

In [5]:
DATA_DIR = '/source/main/data_download/output'

In [6]:
encoders = my_problem.feature_encoders(data_dir=DATA_DIR)

W0813 16:25:12.279611 139831346550528 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/data_generators/text_encoder.py:938: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0813 16:25:12.280572 139831346550528 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/data_generators/text_encoder.py:940: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [7]:
# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    """Input str to features dict, ready for inference"""
    inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
    return {"inputs": batch_inputs}

def decode(integers):
    """List of ints to str"""
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(np.squeeze(integers))

In [8]:
model_name = "transformer"
hparams_set = "transformer_poetry"

In [9]:
hparams = trainer_lib.create_hparams(hparams_set, data_dir=DATA_DIR, problem_name='auto_comment_problem')

In [10]:
my_model = registry.model(model_name)(hparams, Modes.EVAL)

W0813 16:25:14.626328 139831346550528 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/layers/common_layers.py:94: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0813 16:25:14.883188 139831346550528 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/utils/t2t_model.py:243: The name tf.summary.text is deprecated. Please use tf.compat.v1.summary.text instead.



In [11]:
# Copy the pretrained checkpoint locally
ckpt_path = tf.train.latest_checkpoint('/source/main/train/output')
ckpt_path

'/source/main/train/output/model.ckpt-287000'

In [30]:
# Restore and translate!
def translate(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = my_model.infer(encoded_inputs)["outputs"]
        return decode(model_output)

inputs = """__share__ Các mom có ai cho bé uống sữa này cho mình xin rewei nha. Con mình đang uống friso mình định cho bé đổi qua similac mà không biết của nó có ngọt lắm ko, có béo nhiều ko. Cảm ơn các mom nhé __post__"""
outputs = translate(inputs)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)

Inputs: __share__ Các mom có ai cho bé uống sữa này cho mình xin rewei nha. Con mình đang uống friso mình định cho bé đổi qua similac mà không biết của nó có ngọt lắm ko, có béo nhiều ko. Cảm ơn các mom nhé __post__
Outputs:  Bé nhà mình uống friso từ lúc sinh non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non non
